In [ ]:
!pip install pymupdf

   ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
    --------------------------------------- 0.1/3.4 MB 1.3 MB/s eta 0:00:03
   ------- -------------------------------- 0.7/3.4 MB 6.9 MB/s eta 0:00:01
   ------------------- -------------------- 1.6/3.4 MB 11.5 MB/s eta 0:00:01
   ------------------------ --------------- 2.1/3.4 MB 11.3 MB/s eta 0:00:01
   ---------------------------------- ----- 2.9/3.4 MB 13.2 MB/s eta 0:00:01
   ---------------------------------------  3.4/3.4 MB 12.7 MB/s eta 0:00:01
   ---------------------------------------- 3.4/3.4 MB 12.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/24.5 MB ? eta -:--:--
   - -------------------------------------- 0.6/24.5 MB 18.9 MB/s eta 0:00:02
   -- ------------------------------------- 1.5/24.5 MB 23.1 MB/s eta 0:00:01
   --- ------------------------------------ 2.4/24.5 MB 19.2 MB/s eta 0:00:02
   ---- -------------

In [ ]:
import fitz  # PyMuPDF

def create_bounding_boxes(pdf_path):
    doc = fitz.open(pdf_path)

    for page_number in range(doc.page_count):
        page = doc[page_number]
        text_blocks = page.get_text("blocks")

        for block in text_blocks:
            rect = block[0]  # Bounding box coordinates (x0, y0, x1, y1)
            text = block[4]  # Extracted text

            # Print or process bounding boxes as needed
            print(f"Bounding box: {rect}, Text: {text}")

    doc.close()

# Example usage
pdf_path = "C:/Users/shubh/Downloads/resume_dataset/Amit_Kumar resume - AMIT KUMAR.pdf"
create_bounding_boxes(pdf_path)

Bounding box: 222.16165161132812, Text: A B O U T

Bounding box: 222.16162109375, Text: I am a fresher who is extremely motivated to constantly develop my
skills and grow professionally. I am confident in my ability to come up
with interesting ideas. I am a responsible and ambitious student with
excellent time management and always open to learn at the same time.

Bounding box: 222.16162109375, Text: E D U C A T I O N

Bounding box: 222.89175415039062, Text: W O R K  E X P E R I E N C E

Bounding box: 222.16162109375, Text: I DO A SMALL PROJECT IN GRADUATION ON ALGEBRA

Bounding box: 222.16162109375, Text: KL DAV PG COLLEGE ROORKEE, HARIDWAR UTTARAKHAND
PIN CODE: 247667

Bounding box: 222.52670288085938, Text: A C H I E V E M E N T S  A N D  C E R T I F I C A T I O N S

Bounding box: 222.52674865722656, Text: Course on Mobile Repairing

Bounding box: 222.52674865722656, Text: Qualified JAM (Mathematics) Exam

Bounding box: 222.52674865722656, Text: School Topper in Class 10th&12th

Bou

In [ ]:
import os
import fitz  # PyMuPDF

def create_bounding_boxes_for_folder(input_folder, output_folder):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Process each PDF file in the input folder
    for pdf_file in os.listdir(input_folder):
        if pdf_file.endswith(".pdf"):
            pdf_path = os.path.join(input_folder, pdf_file)
            output_text_file = os.path.join(output_folder, f"{os.path.splitext(pdf_file)[0]}.txt")

            create_bounding_boxes(pdf_path, output_text_file)

def create_bounding_boxes(pdf_path, output_text_file):
    doc = fitz.open(pdf_path)

    with open(output_text_file, "w", encoding="utf-8") as text_file:
        for page_number in range(doc.page_count):
            page = doc[page_number]
            text_blocks = page.get_text("blocks")

            for block in text_blocks:
                rect = block[0]  # Bounding box coordinates (x0, y0, x1, y1)
                text = block[4]  # Extracted text

                # Write bounding box and text to the output text file
                #text_file.write(f"Bounding box: {rect}, Text: {text}\n")
                text_file.write(f"{text}\n")

    doc.close()

input_folder = "C:/Users/shubh/Downloads/trial_dataset"
output_folder = "C:/Users/shubh/Downloads/preprocessed_dataset"

create_bounding_boxes_for_folder(input_folder, output_folder)


Added more heuristics

In [ ]:
import fitz  # PyMuPDF

def extract_custom_blocks(pdf_path):
    doc = fitz.open(pdf_path)
    custom_blocks = []

    for page in doc:
        text_blocks = page.get_text("blocks")

        merged_blocks = []
        current_block = {"text": "", "rect": None}

        for block in text_blocks:
            rect = block[0]  # Bounding box coordinates (x0, y0, x1, y1)
            text = block[4]  # Extracted text

            # Check if the block is a heading based on its font size, position, etc.
            is_heading = check_if_heading(block)

            if is_heading:
                # If it's a heading, start a new merged block
                if current_block["text"]:
                    merged_blocks.append(current_block)
                current_block = {"text": text, "rect": rect}
            else:
                # If it's not a heading, append its text to the current merged block
                current_block["text"] += "\n" + text
                current_block["rect"] = (min(rect[0], current_block["rect"][0]),  # Update the bounding box
                                         min(rect[1], current_block["rect"][1]),
                                         max(rect[2], current_block["rect"][2]),
                                         max(rect[3], current_block["rect"][3]))

        if current_block["text"]:
            merged_blocks.append(current_block)

        custom_blocks.extend(merged_blocks)

    doc.close()

    return custom_blocks

def check_if_heading(block):
    # Get the font size and text from the block
    font_size = block[1]
    text = block[4]

    # Check if the text is written in all uppercase or if the font size is larger
    return text.isupper() or font_size > 12  # Adjust the font size threshold as needed


# Example usage
# pdf_path = "path/to/your/file.pdf"
pdf_path = "C:/Users/shubh/Downloads/trial_dataset/Resume_Sagar_Gandhi.pdf"
custom_blocks = extract_custom_blocks(pdf_path)

for block in custom_blocks:
    print(f"Bounding box: {block['rect']}, Text: {block['text']}")


Bounding box: 55.91999816894531, Text: Sagar Gandhi                                                                                     Sagar.Gandhi.In@outlook.com   

Bounding box: 55.20000076293945, Text: Senior Associate, Nagarro, Gurgaon                                                +91-9990212179   
--------------------------------------------------------------------------------------------------------------------- 
Career Objective:   
To secure a significant position where I can both learn and execute my engineering and analytical skills 
for advancement of company and my career.   
   
Summary 

Bounding box: 73.2239990234375, Text: • 
Strong experience in C++ and knowledge of standard library, STL containers, and algorithms.  

Bounding box: 73.2239990234375, Text: • 
Having almost 3 years and 10 months of experience in software development.     

Bounding box: 73.2239990234375, Text: • 
Designed, developed, and tested software applications meeting deadlines and budgets. 

Bo